In [33]:
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

from sklearn.utils import class_weight

from datasets.load_moonboard import load_moonboard

In [3]:
class MoonBoardDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train = True):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train = train
        
        (self.x_train, self.y_train), (self.x_test, self.y_test) = load_moonboard()
        self.x_train = self.x_train.reshape(-1,18,11).astype(float)
        self.x_test = self.x_test.reshape(-1,18,11).astype(float)
        self.y_train = self.y_train.reshape(-1,1).astype(int)
        self.y_test = self.y_test.reshape(-1,1).astype(int)
        #self.y_train = np.eye(17)[self.y_train]
        #self.y_test = np.eye(17)[self.y_test]

    def __len__(self):
        if self.train:
            return len(self.x_train)
        else:
            return len(self.x_test)

    def __getitem__(self, idx):
        if self.train:
            return self.x_train[idx], self.y_train[idx]
        else:
            return self.x_test[idx], self.y_test[idx]


    # prepare data

dataset = MoonBoardDataset(train = True)

train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=128, shuffle=True)
    
test_loader = torch.utils.data.DataLoader(
    MoonBoardDataset(train = False),
    batch_size=32, shuffle=True)

In [68]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 2, 1)
        self.conv2 = nn.Conv2d(20, 50, 2, 1)
        self.fc1 = nn.Linear(11*18, 500)
        self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(500, 500)
        self.fc4 = nn.Linear(500, 500)
        self.fc5 = nn.Linear(500, 500)
        self.fc6 = nn.Linear(500, 500)
        self.fc7 = nn.Linear(500, 500)
        self.fc8 = nn.Linear(500, 17)

    def forward(self, x):
        x = x.view(-1, 11*18)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        x = self.fc7(x)
        x = self.fc8(x)
        return x

In [72]:
torch.manual_seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.7)



cuda


In [73]:
def acc(model, test=True):
    correct = 0
    total = 0
    with torch.no_grad():
        if test:
            loader = test_loader
        else:
            loader = train_loader
        for data in loader:
            images, labels = data
            #print(images[0])
            outputs = model(images.to(device, dtype=torch.float32))

            outputs = outputs.view((-1,17))            
            #print(outputs)
            _, predicted = torch.max(outputs.data, 1)

            
            labels = labels.view((-1))
            total += labels.size(0)
            correct += (predicted == labels.to(device, dtype=torch.long)).sum().item()


    print('Accuracy of the network on the 10000 test images: %f' % (
        100 * correct / total))
    
def acc_class(model):
    class_correct = list(0. for i in range(18))
    class_total = list(0. for i in range(18))
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images.to(device, dtype=torch.float32))
            outputs = outputs.view((-1,17))

            _, predicted = torch.max(outputs.data, 1)

            labels = labels.view((-1))
            c = (predicted == labels.to(device, dtype=torch.long))
            for i in range(len(labels)):
                label = labels[i].item()
                class_correct[label] += c[i].item()
                class_total[label] += 1


    for i in range(1,18):
        if class_total[i] != 0:
            print('Accuracy of %5s : %2d %%' % (
                i, 100 * class_correct[i] / class_total[i]))

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device, dtype=torch.float32), target.to(device, dtype=torch.long)
        optimizer.zero_grad()
        output = model(data)
        target = target.view((-1))
        loss = F.cross_entropy(output, target, weight=torch.tensor(weights).to(device, dtype = torch.float))
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
        



In [74]:
for i in range(100):
    train(model, device, train_loader, optimizer, i)
    acc(model)
    
acc_class(model)

Train Epoch: 0 [0/9524 (0%)]	Loss: 2.844693
Train Epoch: 0 [1280/9524 (13%)]	Loss: 2.831206
Train Epoch: 0 [2560/9524 (27%)]	Loss: 2.828385
Train Epoch: 0 [3840/9524 (40%)]	Loss: 2.819373
Train Epoch: 0 [5120/9524 (53%)]	Loss: 2.811541
Train Epoch: 0 [6400/9524 (67%)]	Loss: 2.834770
Train Epoch: 0 [7680/9524 (80%)]	Loss: 2.804019
Train Epoch: 0 [8960/9524 (93%)]	Loss: 2.790131
Accuracy of the network on the 10000 test images: 15.155332
Train Epoch: 1 [0/9524 (0%)]	Loss: 2.781981
Train Epoch: 1 [1280/9524 (13%)]	Loss: 2.785842
Train Epoch: 1 [2560/9524 (27%)]	Loss: 2.799428
Train Epoch: 1 [3840/9524 (40%)]	Loss: 2.768166
Train Epoch: 1 [5120/9524 (53%)]	Loss: 2.754759
Train Epoch: 1 [6400/9524 (67%)]	Loss: 2.773578
Train Epoch: 1 [7680/9524 (80%)]	Loss: 2.743246
Train Epoch: 1 [8960/9524 (93%)]	Loss: 2.791187
Accuracy of the network on the 10000 test images: 15.155332
Train Epoch: 2 [0/9524 (0%)]	Loss: 2.730373
Train Epoch: 2 [1280/9524 (13%)]	Loss: 2.724842
Train Epoch: 2 [2560/9524 (2

In [11]:
(x_train, y_train), (x_test, y_test) = load_moonboard()


In [75]:
count_train = [0]*17
for label in y_train:
    count_train[label] +=1
print(count_train)
s = sum(count_train)
weights = [s/i  if i != 0 else 0 for i in count_train]
print(weights)

count_test = [0]*17
for label in y_test:
    count_test[label] +=1
print(count_test)

print(sum(count_test) + sum(count_train))

[1, 0, 0, 0, 2890, 933, 1358, 1335, 1061, 544, 642, 411, 197, 92, 13, 13, 34]
[9524.0, 0, 0, 0, 3.295501730103806, 10.207931404072884, 7.013254786450663, 7.134082397003746, 8.976437323279924, 17.50735294117647, 14.834890965732088, 23.172749391727493, 48.34517766497462, 103.52173913043478, 732.6153846153846, 732.6153846153846, 280.11764705882354]
[0, 0, 0, 0, 738, 255, 361, 278, 299, 124, 135, 111, 51, 18, 6, 1, 5]
11906
